# A (Possible?) Code for Identifying Clusters

First, we import Acqu (which parses the events?) and Timepix (which decodes the data??), as well as numpy, ROOT, and plotting classes of ROOT which allow us to organize and graph the data.

In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas

Welcome to JupyROOT 6.16/00


Next, we import the data that was collected and open that data file.

In [5]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Getting arrays of the data from each detector

In [6]:
TimepixAData = []
TimepixBData = []

def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
        TimepixBData.append(Timepix.Decode(nHitsB,encodedB))       
aq.runFunction(plotTimepix,0,50000)

# do we need to sort the data first???

If we want to/need to, we could loop through and make an array of just a certain value (say ToA):

In [90]:
AData = []
ToAA = []

for i in range(len(TimepixAData)):
    AData.append(TimepixAData[0]['ToA'])
for j in range(len(AData)):
    for k in range(len(AData[j])):
        ToAA.append(AData[j][k])
        
BData = []
ToAB = []

for i in range(len(TimepixBData)):
    BData.append(TimepixBData[0]['ToA'])
for j in range(len(BData)):
    for k in range(len(BData[j])):
        ToAB.append(BData[j][k])


At some point we'll need to process the cluster:

In [16]:
def process_cluster():
    start_index = cl[0] # start index of the 1st cluster hit
    min_time = TimepixAData[0]['ToA'][start_index]# time (of arrival) of the first hit
    #dtime = Null # time differences?
    meanx = 0.0
    meany = 0.0
    
    n = start_index
    while(n<start_index+cn): # loop over the cluster to find the lowest time, etc. (for loop in Ken's code)
        if(TimepixAData[0]['ToA'][n] < min_time): # if the time at this hit is less than the min time we've found
            min_time = TimepixAData[0]['ToA'][n]# then set this lower value to the min time
        meanx+=TimepixAData[0]['x'][n] # adding up all x positions
        meany+=TimepixAData[0]['y'][n] # adding up all y positions
        n+=1
    
    meanx/=cn # divide the sum of positions by number of hits to get the mean
    meany/=cn # same as meanx, but for y
    
    ROOT.enableJSVis()
    
    d2pos = Hist2D(10,0,150,10,0,150)
    d2pos.Fill(meanx,meany)
    mult = Hist(100,0,100)
    mult.Fill(nc)
    
    print(meanx, meany)
    
    c4 = ROOT.TCanvas("my4","The Canvas Title",1000,1000)
    c4.Divide(2,1)
    c4.cd(1)
    d2pos.Draw("colz")
    c4.cd(2)
    mult.Draw()
    c4.Draw()
    
    
    
    
    
    print("process :)")

Now let's try to identify clusters. Note: I'm not sure how the data is organized

In [19]:
cl = [None]*100 # holds up to 100 hits in a cluster, initialized as None
cn = 0 # count number of hits in current cluster
blen = 0 # length of current timepix3 buffer
b = 0 # running buffer position

blen = 100 #readTimepixBufferFromFile()
while(blen): # read each timepix buffer into timepix3, except it does't work
    nc = 0 # zero the cluster counter - counts the number of clusters
    while(b<blen): # for every hit in the buffer (buffer position is less than the length of the buffer)
        cl[cn] = b # assume this buffer position is the 1st hit in the current cluster, save it to this cluster's array
        cn+=1 # and increment the counter of hits in this cluster
        
        c=b+1 # use c as a temporary buffer position to compare with
        while((c<b+99) and c<blen): # check the next 99 to see if they are in the time window (for loop in Ken's code)
            if(abs(TimepixAData[0]['ToA'][c]-TimepixAData[0]['ToA'][b])<100): # if within 100ns of other event in the cluster
                cl[cn]=c # save the buffer position into this cluster's array
                cn+=1 # increment the counter of hits in this cluster
            else:
                if cl[0]!= None:
                    process_cluster() # do something with this cluster
                b+=cn # move on to check beyond this cluster
                cn = 0 # reset the number of hits for the new cluster
                cl = [None]*100 # reset the array for the new cluster
                nc += 1 # increment the number of clusters found
            c+=1 # increment to the next buffer position
    print(cl)
    print("end")
    break
    
    


(14.0, 25.0)
process :)
(171.0, 115.0)
process :)
(171.0, 115.0)
process :)
(92.0, 113.0)
process :)
(92.0, 113.0)
process :)

/home/clas12/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in uint_scalars
  from ipykernel import kernelapp as app



(93.0, 113.0)
process :)
(93.0, 113.0)
process :)
(153.0, 105.0)
process :)
(148.0, 230.0)
process :)
(181.0, 167.0)
process :)
(98.0, 98.0)
process :)
(99.0, 98.0)
process :)
(230.5, 0.0)
process :)
(230.0, 0.0)
process :)
(192.0, 184.0)
process :)
(251.0, 178.0)
process :)
(251.0, 178.0)
process :)
(167.0, 72.0)
process :)
(214.0, 125.0)
process :)
(217.25, 125.25)
process :)
(218.0, 125.0)
process :)
(203.0, 132.0)
process :)
(152.0, 91.0)
process :)
(153.0, 90.0)
process :)
(153.0, 90.0)
process :)
(185.0, 175.0)
process :)
(118.0, 158.0)
process :)
(118.0, 157.0)
process :)
(143.0, 170.0)
process :)
(119.0, 130.0)
process :)
(120.0, 130.0)
process :)
(156.0, 121.0)
process :)
(156.0, 121.0)
process :)
(157.0, 216.0)
process :)
(157.0, 215.0)
process :)
(150.0, 249.0)
process :)
(142.0, 119.0)
process :)
(143.0, 118.0)
process :)
(143.0, 118.0)
process :)
(136.0, 219.0)
process :)
(136.0, 219.0)
process :)
(244.0, 211.0)
process :)
(244.0, 210.0)
process :)
(151.0, 153.0)
process 

IndexError: list assignment index out of range